# DISEASE OUTBREAK PREDICTION

importing necessary libraries

In [1]:
# importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd # parallel processing for pandas (to read large datasets) fixed memory issue
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from datetime import datetime, timezone
nltk.download('wordnet')
import locationtagger
import spacy

C:\Users\kedar\AppData\Local\Programs\Python\Python310\lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 13.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kedar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Reading Datasets

In [2]:
# reading comments dataset (do not have enough computational power and memory space to read the entire dataset)
data_comments = pd.read_csv('C:\\Users\\kedar\\OneDrive\\Desktop\\ML mini project\\DiseaseOutbreak_Dataset\\the-reddit-covid-dataset-comments.csv', nrows=1000000)

In [3]:
#checking whether the data loaded
data_comments.head()

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score
0,comment,hi1vsag,2riyy,nova,False,1635206399,https://old.reddit.com/r/nova/comments/qfs53d/...,When you scheduled your booster with CVS does ...,0.0000,2
1,comment,hi1vs7i,2qhov,vancouver,False,1635206397,https://old.reddit.com/r/vancouver/comments/qf...,Didn't stop prices there though. New Zealand a...,0.1887,32
2,comment,hi1vs5n,2qwzb,pregnant,False,1635206397,https://old.reddit.com/r/pregnant/comments/qfs...,I’m just waiting until after pregnancy to get ...,0.6720,1
3,comment,hi1vs5v,2qixm,startrek,False,1635206397,https://old.reddit.com/r/startrek/comments/qft...,*The first duty of every Starfleet officer is ...,0.9562,1
4,comment,hi1vs0l,2qsf3,ontario,False,1635206395,https://old.reddit.com/r/ontario/comments/qfkj...,Compare BC to Ontario for COVID. It's even mor...,0.0000,-2


In [4]:
# reading posts dataset
data_posts = pd.read_csv('C:\\Users\\kedar\\OneDrive\\Desktop\\ML mini project\\DiseaseOutbreak_Dataset\\the-reddit-covid-dataset-posts.csv',nrows=1000000)

In [5]:
# checking whetehr the dataset is loaded
data_posts.head()

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score
0,post,qftvfn,3h9d4,autonewspaper,False,1635206379,https://old.reddit.com/r/AutoNewspaper/comment...,washingtontimes.com,https://www.washingtontimes.com/news/2021/oct/...,NaN,"[Politics] - Kay Ivey, Alabama governor, direc...",1
1,post,qftusg,2x4yx,coronavirus,False,1635206322,https://old.reddit.com/r/Coronavirus/comments/...,twitter.com,NaN,[deleted],More Floridians dead from COVID than Vietnam W...,1
2,post,qftugv,3h9d4,autonewspaper,False,1635206292,https://old.reddit.com/r/AutoNewspaper/comment...,nzherald.co.nz,https://www.nzherald.co.nz/nz/news/article.cfm...,NaN,[NZ] - Covid-19 coronavirus Delta outbreak: La...,1
3,post,qftuge,3h9d4,autonewspaper,False,1635206291,https://old.reddit.com/r/AutoNewspaper/comment...,nzherald.co.nz,https://www.nzherald.co.nz/nz/news/article.cfm...,NaN,[NZ] - Covid 19 Delta outbreak: Man arrested a...,1
4,post,qftucd,4egz78,cointuta,False,1635206281,https://old.reddit.com/r/CoinTuta/comments/qft...,cointuta.com,https://www.cointuta.com/covid-19-more-likely-...,NaN,Covid-19 more likely to cause neurological iss...,1


In [6]:
data_stats = pd.read_csv('WHO-COVID-19-global-data.csv')
data_stats.head()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,AF,Afghanistan,EMRO,0,0,0,0
1,2020-01-04,AF,Afghanistan,EMRO,0,0,0,0
2,2020-01-05,AF,Afghanistan,EMRO,0,0,0,0
3,2020-01-06,AF,Afghanistan,EMRO,0,0,0,0
4,2020-01-07,AF,Afghanistan,EMRO,0,0,0,0


In [7]:
nan_stats = data_stats.isna().sum()
print('Nan values in column of Comments dataset: ',nan_stats)
data_stats = data_stats.drop(columns = 'Country_code')
data_stats.head()

Nan values in column of Comments dataset:  Date_reported           0
Country_code         1407
Country                 0
WHO_region              0
New_cases               0
Cumulative_cases        0
New_deaths              0
Cumulative_deaths       0
dtype: int64


,Date_reported,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,Afghanistan,EMRO,0,0,0,0
1,2020-01-04,Afghanistan,EMRO,0,0,0,0
2,2020-01-05,Afghanistan,EMRO,0,0,0,0
3,2020-01-06,Afghanistan,EMRO,0,0,0,0
4,2020-01-07,Afghanistan,EMRO,0,0,0,0


Data Preprocessing

In [8]:
# analysing missing values
nan_comments = data_comments.isna().sum()
nan_posts = data_posts.isna().sum()
print('Nan values in column of Comments dataset: ',nan_comments)
print('Nan values in each column of Posts dataset: ',nan_posts)

Nan values in column of Comments dataset:  type                  0
id                    0
subreddit.id          0
subreddit.name        0
subreddit.nsfw        0
created_utc           0
permalink             0
body                  0
sentiment         37937
score                 0
dtype: int64
Nan values in each column of Posts dataset:  type                   0
id                     0
subreddit.id           0
subreddit.name         0
subreddit.nsfw         0
created_utc            0
permalink              0
domain                 4
url               257459
selftext          742541
title                  0
score                  0
dtype: int64


In [9]:
# lots of missing values in comments dataset - sentiment column
# lots of missing values in posts dataset - url and selftext columns
# dropping these columns
data_comments = data_comments.drop(columns = 'sentiment')
data_posts = data_posts.drop(columns = ['url','selftext'])

#dropping type,id, subreddit.id, subreddit.name,subreddit.nsfw, permalink in comments dataset since they aren't relevant to the prediction of an outbreak
# similarly dropping type,id,subreddit.id, subreddit.name,subreddit.nsfw,permalink,domain
data_comments = data_comments.drop(columns = ['type','id','subreddit.id', 'subreddit.name','subreddit.nsfw', 'permalink'])
data_posts = data_posts.drop(columns = ['type','id','subreddit.id', 'subreddit.name','subreddit.nsfw','permalink','domain'])

In [10]:
# checking if chenges were made
print('Columns in comments dataset: ',data_comments.columns)
print('Columns in posts dataset: ',data_posts.columns)

# re-analysing missing values
nan_comments = data_comments.isna().sum()
nan_posts = data_posts.isna().sum()
print('Nan values in column of Comments dataset: ',nan_comments)
print('Nan values in each column of Posts dataset: ',nan_posts)

Columns in comments dataset:  Index(['created_utc', 'body', 'score'], dtype='object')
Columns in posts dataset:  Index(['created_utc', 'title', 'score'], dtype='object')
Nan values in column of Comments dataset:  created_utc    0
body           0
score          0
dtype: int64
Nan values in each column of Posts dataset:  created_utc    0
title          0
score          0
dtype: int64


In [11]:
# converting all utc objects to date-time objects
data_comments['created_utc'] = pd.to_datetime(data_comments['created_utc'], unit='s', utc=True)
data_posts['created_utc'] = pd.to_datetime(data_posts['created_utc'], unit='s', utc=True)

In [12]:
# checking 
print('datatype of created utc column in comments dataset: ',data_comments['created_utc'].dtype)
print('datatype of created utc column in posts dataset: ',data_posts['created_utc'].dtype)

datatype of created utc column in comments dataset:  datetime64[ns, UTC]
datatype of created utc column in posts dataset:  datetime64[ns, UTC]


In [13]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kedar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kedar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stop words
    tokens = [word for word in tokens if word not in stop_words]
    # Stemming (optional)
    # tokens = [ps.stem(word) for word in tokens]
    # Join the tokens back into a string
    text = ' '.join(tokens)
    return text
data_comments['body'] = data_comments['body'].apply(preprocess_text)
data_posts['title'] = data_posts['title'].apply(preprocess_text)

In [15]:
# def extract_locations(text):
#     # Check if the input text is not NaN
#     if not pd.isna(text):
#         place_entity = locationtagger.find_locations(text=text)
        
#         # Extract countries, regions, and cities as strings
#         countries = ', '.join(place_entity.countries) if place_entity.countries else ''
# #         regions = ', '.join(place_entity.regions) if place_entity.regions else ''
#         cities = ', '.join(place_entity.cities) if place_entity.cities else ''

#         # Combine countries, regions, and cities into a single string
#         result = f'{countries}, {regions}, {cities}'.strip(', ')

#         # Print statements for checking
#         print(f"Input: {text}")
#         print(f"Countries: {countries}")
#         print(f"Regions: {regions}")
#         print(f"Cities: {cities}")
#         print(f"Result: {result}")

#         return result

#     return 'unknown location'

# # Apply the function to your single column and create a new 'locations' column
# data_comments['locations'] = data_comments['body'].apply(extract_locations)


In [16]:
data_comments.head()
data_posts.head()

,created_utc,title,score
0,2021-10-25 23:59:39+00:00,politics kay ivey alabama governor directs age...,1
1,2021-10-25 23:58:42+00:00,floridians dead covid vietnam war video,1
2,2021-10-25 23:58:12+00:00,nz covid19 coronavirus delta outbreak latest c...,1
3,2021-10-25 23:58:11+00:00,nz covid 19 delta outbreak man arrested trying...,1
4,2021-10-25 23:58:01+00:00,covid19 likely cause neurological issues vacci...,1


In [6]:
from geopy.geocoders import Nominatim
from nltk.corpus import words
from nltk.tokenize import word_tokenize
import nltk
import re

# Download NLTK words corpus (if not already downloaded)
nltk.download('words')

def location_tagger(text):
    geolocator = Nominatim(user_agent="my_geocoder")
    english_words = set(words.words())

    text = text
    tokens = re.findall(r'\b\w+\b', text)
    detected_locations = []

    # Iterate through tokens and check for potential locations
    for token in tokens:
        # Check if the token is a valid English word
        if token.lower() not in english_words:
            # Perform spell-checking using NLTK and suggest corrections
            suggestions = [word for word in english_words if nltk.edit_distance(token.lower(), word) < 2]
            print(f"The word '{token}' is likely misspelled. Suggestions: {suggestions}")

            # Choose the suggestion with the smallest edit distance
            best_suggestion = min(suggestions, key=lambda x: nltk.edit_distance(token.lower(), x))
            location = geolocator.geocode(best_suggestion)
        else:
            # Perform geocoding to get the location information
            location = geolocator.geocode(token)

        # Check if a location was found
        if location:
            print("location: ")
            detected_locations.append({'location': location})
        else:
            location = 'unknown location'
            detected_locations.append({'location': location})

    print(detected_locations)
    return detected_locations

# Example usage
location_tagger("Didn't stop prices there though. New Zealand a...")


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\kedar\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


The word 'Didn' is likely misspelled. Suggestions: ['did', 'dian', 'din', 'didna', 'didnt']
location: 
location: 
location: 
The word 'prices' is likely misspelled. Suggestions: ['price', 'priced', 'pricer', 'preces', 'pricks']
location: 
location: 
location: 
location: 
The word 'Zealand' is likely misspelled. Suggestions: ['lealand']
location: 
location: 
[{'location': Location(Did, Ilovik, Grad Mali Lošinj, Primorsko-goranska županija, 51552, Hrvatska, (44.4506199, 14.547613, 0.0))}, {'location': Location(Thüringen, Deutschland, (50.9014721, 11.0377839, 0.0))}, {'location': Location(Stop, Bovec, Slovenija, (46.4369086, 13.6728474, 0.0))}, {'location': Location(Price County, Wisconsin, United States, (45.676627, -90.368792, 0.0))}, {'location': Location(there, Uhembo, Central Alego ward, Alego Usonga, Siaya, Kenya, (0.13559495, 34.26772878953496, 0.0))}, {'location': Location(Campus Cut Though, Heslington, York, England, United Kingdom, (53.94709015, -1.0383845379306207, 0.0))}, {'lo

[{'location': Location(Did, Ilovik, Grad Mali Lošinj, Primorsko-goranska županija, 51552, Hrvatska, (44.4506199, 14.547613, 0.0))},
 {'location': Location(Thüringen, Deutschland, (50.9014721, 11.0377839, 0.0))},
 {'location': Location(Stop, Bovec, Slovenija, (46.4369086, 13.6728474, 0.0))},
 {'location': Location(Price County, Wisconsin, United States, (45.676627, -90.368792, 0.0))},
 {'location': Location(there, Uhembo, Central Alego ward, Alego Usonga, Siaya, Kenya, (0.13559495, 34.26772878953496, 0.0))},
 {'location': Location(Campus Cut Though, Heslington, York, England, United Kingdom, (53.94709015, -1.0383845379306207, 0.0))},
 {'location': Location(New, Owen County, Kentucky, United States, (38.4278468, -84.8105037, 0.0))},
 {'location': Location(Lealand, Clarksville, Montgomery County, Middle Tennessee, Tennessee, 37042, United States, (36.5648113, -87.4090042366841, 0.0))},
 {'location': Location(A, 120, Gale Place, Bronx County, The Bronx, City of New York, New York, 10463, U

In [8]:
import re
import nltk
import locationtagger

# Download NLTK words corpus (if not already downloaded)
nltk.download('words')

def location_tagger(text):
    english_words = set(nltk.corpus.words.words())

    text = text
    tokens = re.findall(r'\b\w+\b', text)
    detected_locations = []

    # Iterate through tokens and check for potential locations
    for token in tokens:
        # Check if the token is a valid English word
        if token.lower() not in english_words:
            # Perform spell-checking using NLTK and suggest corrections
            suggestions = [word for word in english_words if nltk.edit_distance(token.lower(), word) < 2]
            print(f"The word '{token}' is likely misspelled. Suggestions: {suggestions}")

            # Choose the suggestion with the smallest edit distance
            best_suggestion = min(suggestions, key=lambda x: nltk.edit_distance(token.lower(), x))
            place_entity = locationtagger.find_locations(text=best_suggestion)
        else:
            # Perform geocoding to get the location information
            place_entity = locationtagger.find_locations(text=token)

        # Extract countries, regions, and cities as strings
        countries = ', '.join(place_entity.countries) if place_entity.countries else ''
        regions = ', '.join(place_entity.regions) if place_entity.regions else ''
        cities = ', '.join(place_entity.cities) if place_entity.cities else ''

        # Combine countries, regions, and cities into a single string
        result = f'{countries}, {regions}, {cities}'.strip(', ')

        # Print statements for checking
        print(f"Input: {token}")
        print(f"Countries: {countries}")
        print(f"Regions: {regions}")
        print(f"Cities: {cities}")
        print(f"Result: {result}")

        detected_locations.append({'location': result})

    print(detected_locations)
    return detected_locations

# Example usage
location_tagger("Didn't stop prices there though. pari a...")

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\kedar\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


The word 'Didn' is likely misspelled. Suggestions: ['did', 'dian', 'din', 'didna', 'didnt']
Input: Didn
Countries: 
Regions: 
Cities: 
Result: 
Input: t
Countries: 
Regions: 
Cities: 
Result: 
Input: stop
Countries: 
Regions: 
Cities: 
Result: 
The word 'prices' is likely misspelled. Suggestions: ['price', 'priced', 'pricer', 'preces', 'pricks']
Input: prices
Countries: 
Regions: 
Cities: 
Result: 
Input: there
Countries: 
Regions: 
Cities: 
Result: 
Input: though
Countries: 
Regions: 
Cities: 
Result: 
Input: pari
Countries: 
Regions: 
Cities: 
Result: 
Input: a
Countries: 
Regions: 
Cities: 
Result: 
[{'location': ''}, {'location': ''}, {'location': ''}, {'location': ''}, {'location': ''}, {'location': ''}, {'location': ''}, {'location': ''}]


[{'location': ''},
 {'location': ''},
 {'location': ''},
 {'location': ''},
 {'location': ''},
 {'location': ''},
 {'location': ''},
 {'location': ''}]

In [10]:
import re
import nltk
import locationtagger

# Download NLTK words corpus (if not already downloaded)
nltk.download('words')

def location_tagger(text):
    english_words = set(nltk.corpus.words.words())

    # Tokenize the text using nltk.word_tokenize to preserve multi-word locations
    tokens = nltk.word_tokenize(text)
    detected_locations = []

    i = 0  # Index for iterating through tokens
    while i < len(tokens):
        token = tokens[i]

        # Check if the token is a valid English word
        if token.lower() not in english_words:
            # Combine consecutive tokens to form a potential multi-word location
            multi_word_token = token
            while i + 1 < len(tokens) and tokens[i + 1].lower() not in english_words:
                i += 1
                multi_word_token += f' {tokens[i]}'

            # Perform spell-checking using NLTK and suggest corrections
            suggestions = [word for word in english_words if nltk.edit_distance(multi_word_token.lower(), word) < 2]
            
            # Check if there are suggestions before finding the best one
            if suggestions:
                # Choose the suggestion with the smallest edit distance
                best_suggestion = min(suggestions, key=lambda x: nltk.edit_distance(multi_word_token.lower(), x))
                place_entity = locationtagger.find_locations(text=best_suggestion)
            else:
                place_entity = locationtagger.find_locations(text=multi_word_token)
        else:
            # Perform geocoding to get the location information
            place_entity = locationtagger.find_locations(text=token)

        # Extract countries, regions, and cities as strings
        countries = ', '.join(place_entity.countries) if place_entity.countries else ''
        regions = ', '.join(place_entity.regions) if place_entity.regions else ''
        cities = ', '.join(place_entity.cities) if place_entity.cities else ''

        # Combine countries, regions, and cities into a single string
        result = f'{countries}, {regions}, {cities}'.strip(', ')

        # Print statements for checking
        print(f"Input: {token}")
        print(f"Countries: {countries}")
        print(f"Regions: {regions}")
        print(f"Cities: {cities}")
        print(f"Result: {result}")

        detected_locations.append({'location': result})

        i += 1

    print(detected_locations)
    return detected_locations

# Example usage
location_tagger("Didn't stop prices there though. New Zealand a...")


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\kedar\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


Input: Did
Countries: 
Regions: 
Cities: 
Result: 
Input: n't
Countries: 
Regions: 
Cities: 
Result: 
Input: stop
Countries: 
Regions: 
Cities: 
Result: 
Input: prices
Countries: 
Regions: 
Cities: 
Result: 
Input: there
Countries: 
Regions: 
Cities: 
Result: 
Input: though
Countries: 
Regions: 
Cities: 
Result: 
Input: .
Countries: 
Regions: 
Cities: 
Result: 
Input: New
Countries: 
Regions: 
Cities: 
Result: 
Input: Zealand
Countries: 
Regions: 
Cities: 
Result: 
Input: a
Countries: 
Regions: 
Cities: 
Result: 
Input: ...
Countries: 
Regions: 
Cities: 
Result: 
[{'location': ''}, {'location': ''}, {'location': ''}, {'location': ''}, {'location': ''}, {'location': ''}, {'location': ''}, {'location': ''}, {'location': ''}, {'location': ''}, {'location': ''}]


[{'location': ''},
 {'location': ''},
 {'location': ''},
 {'location': ''},
 {'location': ''},
 {'location': ''},
 {'location': ''},
 {'location': ''},
 {'location': ''},
 {'location': ''},
 {'location': ''}]

In [11]:
import re
import nltk
import locationtagger

# Download NLTK words corpus (if not already downloaded)
nltk.download('words')

def location_tagger(text):
    english_words = set(nltk.corpus.words.words())

    text = text
    tokens = re.findall(r'\b\w+\b', text)
    detected_locations = []

    # Iterate through tokens and check for potential locations
    for token in tokens:
        # Check if the token is a valid English word
        if token.lower() not in english_words:
            # Perform spell-checking using NLTK and suggest corrections
            suggestions = [word for word in english_words if nltk.edit_distance(token.lower(), word) < 2]

            if suggestions:
                # Choose the suggestion with the smallest edit distance
                best_suggestion = min(suggestions, key=lambda x: nltk.edit_distance(token.lower(), x))
                place_entity = locationtagger.find_locations(text=best_suggestion)
            else:
                # If no suggestions, use the original token for location tagging
                place_entity = locationtagger.find_locations(text=token)
        else:
            # Perform geocoding to get the location information
            place_entity = locationtagger.find_locations(text=token)

        # Extract countries, regions, and cities as strings
        countries = ', '.join(place_entity.countries) if place_entity.countries else ''
        regions = ', '.join(place_entity.regions) if place_entity.regions else ''
        cities = ', '.join(place_entity.cities) if place_entity.cities else ''

        # Combine countries, regions, and cities into a single string
        result = f'{countries}, {regions}, {cities}'.strip(', ')

        # Print statements for checking
        print(f"Input: {token}")
        print(f"Countries: {countries}")
        print(f"Regions: {regions}")
        print(f"Cities: {cities}")
        print(f"Result: {result}")

        detected_locations.append({'location': result})

    print(detected_locations)
    return detected_locations

# Example usage
location_tagger("Didn't stop prices there though. pari a...")


[nltk_data] Downloading package words to
[nltk_data]     C:\Users\kedar\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


Input: Didn
Countries: 
Regions: 
Cities: 
Result: 
Input: t
Countries: 
Regions: 
Cities: 
Result: 
Input: stop
Countries: 
Regions: 
Cities: 
Result: 
Input: prices
Countries: 
Regions: 
Cities: 
Result: 
Input: there
Countries: 
Regions: 
Cities: 
Result: 
Input: though
Countries: 
Regions: 
Cities: 
Result: 
Input: pari
Countries: 
Regions: 
Cities: 
Result: 
Input: a
Countries: 
Regions: 
Cities: 
Result: 
[{'location': ''}, {'location': ''}, {'location': ''}, {'location': ''}, {'location': ''}, {'location': ''}, {'location': ''}, {'location': ''}]


[{'location': ''},
 {'location': ''},
 {'location': ''},
 {'location': ''},
 {'location': ''},
 {'location': ''},
 {'location': ''},
 {'location': ''}]

In [12]:
!pip install pyspellchecker


     ---------------------------------------- 3.4/3.4 MB 2.1 MB/s eta 0:00:00


DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\kedar\AppData\Local\programs\Python\Python310\python.exe -m pip install --upgrade pip


In [16]:
import re
from spellchecker import SpellChecker
import locationtagger

def location_tagger(text):
    spell = SpellChecker()
    text = text
    tokens = re.findall(r'\b\w+\b', text)
    detected_locations = []

    for token in tokens:
        if not spell.correction(token).lower() == token.lower():
            corrected_token = spell.correction(token)
            place_entity = locationtagger.find_locations(text=corrected_token)
        else:
            place_entity = locationtagger.find_locations(text=token)

        # Extract countries, regions, and cities as strings
        countries = ', '.join(place_entity.countries) if place_entity.countries else ''
        regions = ', '.join(place_entity.regions) if place_entity.regions else ''
        cities = ', '.join(place_entity.cities) if place_entity.cities else ''

        # Combine countries, regions, and cities into a single string
        result = f'{countries}, {regions}, {cities}'.strip(', ')

        # Print statements for checking
        print(f"Input: {token}")
        print(f"Countries: {countries}")
        print(f"Regions: {regions}")
        print(f"Cities: {cities}")
        print(f"Result: {result}")

        detected_locations.append({'location': result})

    print(detected_locations)
    return detected_locations

# Example usage
location_tagger("in the city of paris")


Input: in
Countries: 
Regions: 
Cities: 
Result: 
Input: the
Countries: 
Regions: 
Cities: 
Result: 
Input: city
Countries: 
Regions: 
Cities: 
Result: 
Input: of
Countries: 
Regions: 
Cities: 
Result: 
Input: paris
Countries: 
Regions: Paris
Cities: Paris
Result: Paris, Paris
[{'location': ''}, {'location': ''}, {'location': ''}, {'location': ''}, {'location': 'Paris, Paris'}]


[{'location': ''},
 {'location': ''},
 {'location': ''},
 {'location': ''},
 {'location': 'Paris, Paris'}]